## Pipeline to create cons_temp_df

and variants depending on model run time

In [17]:
func_region = "Auvergne-Rhône-Alpes"
func_target_day = "2025-04-08"
func_model = "M48"
func_run_time = "02:00:00"

In [18]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import mlflow
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
import mlflow.xgboost
from dictionaries import weather_stations, region_abbr_dict, region_abbr_caps_dict, run_time_dict, model_delta, holiday_zones, prediction_timeframes

from vacances_scolaires_france import SchoolHolidayDates

In [19]:
# Define your function early on (in one cell)
def prepare_pipeline_inputs(region: str, chosen_day_str: str, model: str, run_time: str):
    
    # Parse chosen day as datetime
    chosen_day = datetime.strptime(chosen_day_str, "%Y-%m-%d")

    # Region
    Région = region
       
    # Build consumption API where clause for the 5 days prior
    num_days = 6
    consumption_dates = [(chosen_day - timedelta(days=i)).strftime("%Y-%m-%d") 
                         for i in range(num_days, 0, -1)]
    
    consumption_where = f'libelle_region:"{region}" AND (' + " OR ".join(
        f'date:"{d}"' for d in consumption_dates
    ) + ')'
    
    # Build consumption API where clause for the past, target, and next day
    num_days = 3
    consumption_dates_2 = [(chosen_day + timedelta(days=i-1)).strftime("%Y-%m-%d") 
                         for i in range(num_days)]
    
    consumption_where_2 = f'libelle_region:"{region}" AND (' + " OR ".join(
        f'date:"{d}"' for d in consumption_dates_2
    ) + ')'

    # For temperature API, target date is 5 days before chosen day
    target_date = (chosen_day - timedelta(days=5)).strftime("%Y-%m-%d")
    
    # Load weather stations from a dictionary file (assumes file "weather_stations.dict" exists)
    stations_for_region = weather_stations.get(region, [])
        
    # Look up the timeframe for the given model/run_time pair
    timeframe = prediction_timeframes.get((model, run_time))
    if timeframe is None:
        raise ValueError(f"No timeframe defined for model {model} at run time {run_time}")
    
    # Build start and end datetime objects using chosen_day's date.
    start_dt = datetime.combine(chosen_day.date(), datetime.strptime(timeframe["start"], "%H:%M:%S").time())
    end_dt = datetime.combine(chosen_day.date(), datetime.strptime(timeframe["end"], "%H:%M:%S").time())

    # Create the expected timestamps for the prediction placeholder
    expected_timestamps = pd.date_range(start=start_dt, end=end_dt, freq="15min")

    delta = model_delta.get(model)

    deltatime = timedelta(hours=delta)

    # Dictionary for region abbreviations
    region_abbr = region_abbr_dict.get(region, "NA")

   #Dictionary for capitalized region abbreviations
    region_abbr_caps = region_abbr_caps_dict.get(region, "NA")

    #Dictionary for run_time abbreviations
    run_time_abbr = run_time_dict.get(run_time, "NA")
   
    return {
        "chosen_day": chosen_day,
        "consumption_where": consumption_where,
        "consumption_dates": consumption_dates,
        "target_date": target_date,
        "stations_for_region": stations_for_region,
        "region_abbr": region_abbr,
        "region_caps": region_abbr_caps,
        "Région": Région,
        "consumption_where_2": consumption_where_2,
        "consumption_dates_2": consumption_dates_2,
        "model": delta,
        "run_time": run_time,
        "run_time_abbr": run_time_abbr,
        "prediction_timestamps": expected_timestamps,
        "first_row": start_dt,
        "last_row": end_dt,
        "deltatime": deltatime
    }


In [20]:
# Provide your inputs here
inputs = prepare_pipeline_inputs(func_region, func_target_day, func_model, func_run_time)

region_lwrc = inputs["region_abbr"]                          
region_caps = inputs["region_caps"]
date_str = inputs["chosen_day"].strftime("%m-%d")
run_time = inputs["run_time_abbr"]
month = inputs["chosen_day"].strftime("%B")
day = inputs["chosen_day"].day
year = inputs["chosen_day"].year
weekday = inputs["chosen_day"].strftime("%A")
model = inputs["model"]
chosen_day = inputs["chosen_day"]

## Creating df_consumption

### Via API Call

In [21]:
# Define the endpoint URL for the eco2mix-regional-tr dataset
url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr/records"

# Build the query parameters using a 'where' filter.
params = {
    "limit": 100,
    "where": inputs["consumption_where"],
    "select": "date,heure,date_heure,libelle_region,consommation",
    "timezone": "UTC",
    "include_links": "false",
    "include_app_metas": "false"
}

# Set the request headers
headers = {
    "accept": "application/json; charset=utf-8"
}

# Make the GET request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    consumption_json = response.json()  # Convert response to JSON dictionary
    print(json.dumps(consumption_json, indent=2, ensure_ascii=False))
else:
    print(f"Error: HTTP {response.status_code}")
    print(response.text)


{
  "total_count": 576,
  "results": [
    {
      "date": "2025-04-05",
      "heure": "01:45",
      "date_heure": "2025-04-04T23:45:00+00:00",
      "libelle_region": "Auvergne-Rhône-Alpes",
      "consommation": 5850
    },
    {
      "date": "2025-04-05",
      "heure": "03:00",
      "date_heure": "2025-04-05T01:00:00+00:00",
      "libelle_region": "Auvergne-Rhône-Alpes",
      "consommation": 5758
    },
    {
      "date": "2025-04-05",
      "heure": "05:45",
      "date_heure": "2025-04-05T03:45:00+00:00",
      "libelle_region": "Auvergne-Rhône-Alpes",
      "consommation": 5452
    },
    {
      "date": "2025-04-05",
      "heure": "06:30",
      "date_heure": "2025-04-05T04:30:00+00:00",
      "libelle_region": "Auvergne-Rhône-Alpes",
      "consommation": 5642
    },
    {
      "date": "2025-04-05",
      "heure": "07:45",
      "date_heure": "2025-04-05T05:45:00+00:00",
      "libelle_region": "Auvergne-Rhône-Alpes",
      "consommation": 5873
    },
    {
      "dat

In [22]:
# Suppose you have loaded your JSON into a variable called consumption_json
# For example:
# consumption_json = json.loads(your_json_string)

# Create a DataFrame from the "results" list
df_consumption = pd.DataFrame(consumption_json["results"])

# Use the "date_heure" field as the timestamp. Convert it to datetime.
df_consumption["datetime"] = pd.to_datetime(df_consumption["date_heure"])

# Select only the needed columns (e.g., datetime, consommation, and optionally region)
df_consumption = df_consumption[["datetime", "consommation", "libelle_region"]].copy()

#Rename columns
df_consumption.rename(columns={"libelle_region": "Région", "consommation": "Consommation (MW)", "datetime": "Datetime"}, inplace=True)

# Sort by datetime
df_consumption.sort_values("Datetime", inplace=True)

# Convert from UTC to Paris time (CET/CEST)
df_consumption["Datetime"] = df_consumption["Datetime"].dt.tz_convert("Europe/Paris")

df_consumption.set_index("Datetime", inplace=True)

# Resample the data to 15-minute intervals and interpolate linearly
df_consumption = df_consumption.resample("15min").interpolate(method="linear")

# Reset the index so that 'Datetime' is again a column
df_consumption.reset_index(inplace=True)

# Fill NaN values in "Région" column using forward fill
df_consumption["Région"] = df_consumption["Région"].ffill()

C:\Users\Henri\AppData\Local\Temp\ipykernel_12800\1104463976.py:26: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_consumption = df_consumption.resample("15min").interpolate(method="linear")


In [23]:
df_consumption.describe

<bound method NDFrame.describe of                      Datetime  Consommation (MW)                Région
0   2025-04-02 00:45:00+02:00        6877.000000  Auvergne-Rhône-Alpes
1   2025-04-02 01:00:00+02:00        6865.333333  Auvergne-Rhône-Alpes
2   2025-04-02 01:15:00+02:00        6853.666667  Auvergne-Rhône-Alpes
3   2025-04-02 01:30:00+02:00        6842.000000  Auvergne-Rhône-Alpes
4   2025-04-02 01:45:00+02:00        6861.500000  Auvergne-Rhône-Alpes
..                        ...                ...                   ...
567 2025-04-07 22:30:00+02:00        6680.000000  Auvergne-Rhône-Alpes
568 2025-04-07 22:45:00+02:00        6680.000000  Auvergne-Rhône-Alpes
569 2025-04-07 23:00:00+02:00        6680.000000  Auvergne-Rhône-Alpes
570 2025-04-07 23:15:00+02:00        6680.000000  Auvergne-Rhône-Alpes
571 2025-04-07 23:30:00+02:00        6680.000000  Auvergne-Rhône-Alpes

[572 rows x 3 columns]>

## Pipeline with function + for loop to call most recent electrical data + concatenate into historical database

In [ ]:
import requests
import pandas as pd
import json

def get_regional_consumption(region_name):
    # Filter starting from March 30, 2025
    start_date = "2025-03-30"
    where_clause = f"libelle_region='{region_name}' AND date>='{start_date}'"

    url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr/records"
    params = {
        "limit": 100,  # increased limit
        "where": where_clause,
        "select": "date,heure,date_heure,libelle_region,consommation",
        "timezone": "UTC",
        "include_links": "false",
        "include_app_metas": "false"
    }
    headers = {"accept": "application/json; charset=utf-8"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        print(f"❌ Error {response.status_code} for region {region_name}")
        return pd.DataFrame()  # return empty df

    consumption_json = response.json()
    df = pd.DataFrame(consumption_json["results"])

    if df.empty:
        print(f"⚠️ No data found for region {region_name}")
        return df

    df["Datetime"] = pd.to_datetime(df["date_heure"])
    df = df[["Datetime", "consommation", "libelle_region"]].copy()
    df.rename(columns={"libelle_region": "Région", "consommation": "Consommation (MW)"}, inplace=True)

    df.sort_values("Datetime", inplace=True)
    df["Datetime"] = df["Datetime"].dt.tz_convert("Europe/Paris")
    df.set_index("Datetime", inplace=True)
    df = df.resample("15min").interpolate(method="linear")
    df.reset_index(inplace=True)
    df["Région"] = df["Région"].ffill()

    return df


In [25]:
from dictionaries import region_abbr_dict


In [ ]:

# Load your existing dataset
cons_data_path = "data/cons_temp_2025.csv"
df_all = pd.read_csv(cons_data_path, parse_dates=["Datetime"])
df_all["Datetime"] = df_all["Datetime"].dt.tz_localize("Europe/Paris")


In [26]:

# Gather new data
all_new_data = []

for region in region_abbr_dict.keys():
    print(f"📥 Fetching data for {region}")
    df_region = get_regional_consumption(region)
    if not df_region.empty:
        all_new_data.append(df_region)

# Combine new data
df_new = pd.concat(all_new_data).reset_index(drop=True)


📥 Fetching data for Nouvelle-Aquitaine
❌ Error 400 for region Nouvelle-Aquitaine
📥 Fetching data for Occitanie
❌ Error 400 for region Occitanie
📥 Fetching data for Île-de-France
❌ Error 400 for region Île-de-France
📥 Fetching data for Auvergne-Rhône-Alpes
❌ Error 400 for region Auvergne-Rhône-Alpes
📥 Fetching data for Grand Est
❌ Error 400 for region Grand Est
📥 Fetching data for Normandie
❌ Error 400 for region Normandie
📥 Fetching data for Bretagne
❌ Error 400 for region Bretagne
📥 Fetching data for Provence-Alpes-Côte d'Azur
❌ Error 400 for region Provence-Alpes-Côte d'Azur
📥 Fetching data for Hauts-de-France
❌ Error 400 for region Hauts-de-France
📥 Fetching data for Pays de la Loire
❌ Error 400 for region Pays de la Loire
📥 Fetching data for Centre-Val de Loire
❌ Error 400 for region Centre-Val de Loire
📥 Fetching data for Bourgogne-Franche-Comté
❌ Error 400 for region Bourgogne-Franche-Comté


ValueError: No objects to concatenate

In [ ]:

# Remove old records from March 30 onward
df_all = df_all[df_all["Datetime"] < pd.to_datetime("2025-03-30").tz_localize("Europe/Paris")]

# Combine + sort
df_updated = pd.concat([df_all, df_new]).sort_values("Datetime")

# Save updated dataset
df_updated.to_csv("data/cons_temp_2025.csv", index=False)
print("✅ Consumption dataset updated.")


### Via JSON read

In [11]:
filepath = "C:\\Users\\Henri\\Documents\\GitHub\\Predi_Conso_Elec_Region\\Predi_Conso_Elec_Region\\notebooks\\Region tests\\{}\\cons_{}_{}.json".format(region_caps, region_lwrc, date_str)

with open(filepath, "r", encoding="utf-8") as f:
    df_consumption = json.load(f)


In [12]:
df_consumption = pd.json_normalize(df_consumption, record_path=["results"])

In [13]:
# Use the "date_heure" field as the timestamp. Convert it to datetime.
df_consumption["datetime"] = pd.to_datetime(df_consumption["date_heure"])

# Select only the needed columns (e.g., datetime, consommation, and optionally region)
df_consumption = df_consumption[["datetime", "consommation", "libelle_region"]].copy()

# For the consumption data DataFrame (df_consumption):
df_consumption.rename(columns={"datetime": "Datetime", "consommation": "Consommation (MW)", "libelle_region" : "Région"}, inplace=True)

# Sort by datetime
df_consumption.sort_values("Datetime", inplace=True)

# Then remove the timezone information (e.g., "+00:00")
df_consumption["Datetime"] = df_consumption["Datetime"].dt.tz_convert("Europe/Paris")

df_consumption.set_index("Datetime", inplace=True)


In [14]:
df_consumption.describe

<bound method NDFrame.describe of                            Consommation (MW)                   Région
Datetime                                                             
2025-02-28 00:45:00+01:00               2328  Bourgogne-Franche-Comté
2025-02-28 03:00:00+01:00               2246  Bourgogne-Franche-Comté
2025-02-28 05:00:00+01:00               2366  Bourgogne-Franche-Comté
2025-02-28 05:30:00+01:00               2469  Bourgogne-Franche-Comté
2025-02-28 06:30:00+01:00               2634  Bourgogne-Franche-Comté
...                                      ...                      ...
2025-03-04 15:45:00+01:00               2417  Bourgogne-Franche-Comté
2025-03-04 17:00:00+01:00               2355  Bourgogne-Franche-Comté
2025-03-04 17:45:00+01:00               2442  Bourgogne-Franche-Comté
2025-03-04 21:00:00+01:00               2709  Bourgogne-Franche-Comté
2025-03-04 23:45:00+01:00               2405  Bourgogne-Franche-Comté

[100 rows x 2 columns]>

In [15]:
# Resample the data to 15-minute intervals and interpolate linearly
df_consumption = df_consumption.resample("15min").interpolate(method="linear")

# Reset the index so that 'Datetime' is again a column
df_consumption.reset_index(inplace=True)

# Fill NaN values in "Région" column using forward fill
df_consumption["Région"] = df_consumption["Région"].ffill()

C:\Users\Henri\AppData\Local\Temp\ipykernel_8332\297229946.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_consumption = df_consumption.resample("15min").interpolate(method="linear")


In [16]:
df_consumption.to_csv("cons_{}_{}.csv".format(region_lwrc, date_str), index=False)

### Via CSV read

In [17]:
df_consumption = pd.read_csv("cons_{}_{}.csv".format(region_lwrc, date_str), parse_dates=["Datetime"])

### Via Historical Data

In [3]:
df_consumption = pd.read_csv(r"C:\Users\Henri\Documents\GitHub\Predi_Conso_Elec_Region\Predi_Conso_Elec_Region\data\reg_2025_cons.csv", parse_dates=["Datetime"])

In [4]:
df_consumption["Datetime"] = pd.to_datetime(df_consumption["Datetime"], utc=True)

In [5]:
df_consumption.describe

<bound method NDFrame.describe of                         Datetime  Consommation (MW)                   Région
0      2024-12-31 23:00:00+00:00             9557.0            Île-de-France
1      2024-12-31 23:00:00+00:00             2793.0                 Bretagne
2      2024-12-31 23:00:00+00:00             6478.0       Nouvelle-Aquitaine
3      2024-12-31 23:00:00+00:00             3441.0                Normandie
4      2024-12-31 23:00:00+00:00             2619.0      Centre-Val de Loire
...                          ...                ...                      ...
104827 2025-04-01 21:45:00+00:00                NaN     Auvergne-Rhône-Alpes
104828 2025-04-01 21:45:00+00:00                NaN                Normandie
104829 2025-04-01 21:45:00+00:00                NaN  Bourgogne-Franche-Comté
104830 2025-04-01 21:45:00+00:00                NaN      Centre-Val de Loire
104831 2025-04-01 21:45:00+00:00                NaN            Île-de-France

[104832 rows x 3 columns]>

In [7]:
df_consumption.columns

Index(['Datetime', 'Consommation (MW)', 'Région'], dtype='object')

In [6]:
df_consumption["Datetime"] = df_consumption["Datetime"].dt.tz_convert("Europe/Paris").dt.tz_localize(None)

In [7]:
df_consumption.describe

<bound method NDFrame.describe of                   Datetime  Consommation (MW)                   Région
0      2025-01-01 00:00:00             9557.0            Île-de-France
1      2025-01-01 00:00:00             2793.0                 Bretagne
2      2025-01-01 00:00:00             6478.0       Nouvelle-Aquitaine
3      2025-01-01 00:00:00             3441.0                Normandie
4      2025-01-01 00:00:00             2619.0      Centre-Val de Loire
...                    ...                ...                      ...
104827 2025-04-01 23:45:00                NaN     Auvergne-Rhône-Alpes
104828 2025-04-01 23:45:00                NaN                Normandie
104829 2025-04-01 23:45:00                NaN  Bourgogne-Franche-Comté
104830 2025-04-01 23:45:00                NaN      Centre-Val de Loire
104831 2025-04-01 23:45:00                NaN            Île-de-France

[104832 rows x 3 columns]>

In [20]:
df_consumption.isna().sum()

Datetime               0
Consommation (MW)    620
Région                 0
dtype: int64

## Creating df_merged

### Via API call

In [7]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Example dictionary for weather stations per region.
# For Nouvelle-Aquitaine, we choose four stations.
regional_stations = inputs["stations_for_region"]

# Base URL for the API.
# (In the documentation, the sample uses http://127.0.0.1:8000.
# In your production or test environment, update this base URL accordingly.)
BASE_URL = "https://meteo.comptoir.net/api/synops"

def fetch_station_measures(station_id, target_date, fields="measured_at,t"):
    """
    For a given station ID and target_date (string in YYYY-MM-DD format),
    fetch measures for that date using the API.
    
    Returns a DataFrame with columns 'measured_at' and 't' (temperature).
    """
    # Construct the URL:
    # E.g.: https://meteo.comptoir.net//89642/2025-02-20?fields=measured_at,t
    url = f"{BASE_URL}/{station_id}/{target_date}?fields={fields}"
    
    headers = {
        "Accept": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching station {station_id} for {target_date}: HTTP {response.status_code}")
        return None

    data = response.json()
    
    # Check the JSON structure. Depending on the API, the measures may be nested.
    # Here we assume that the JSON has a top-level key "measures" which is a list of dictionaries,
    # each containing the fields "measured_at" and "t".
    if "measures" not in data:
        print(f"Unexpected data format for station {station_id}: {data}")
        return None

    # Create a DataFrame from the "measures" list.
    df = pd.DataFrame(data["measures"])
    
    # Sometimes the API may nest fields inside another key; adjust accordingly if needed.
    # For example, if the actual fields are inside a sub-dictionary (e.g., "record" or "fields"),
    # you would extract them. For now, we'll assume they are at the top level.
    
    # Convert measured_at to datetime.
    if "measured_at" in df.columns:
        df["measured_at"] = pd.to_datetime(df["measured_at"])
    else:
        print(f"'measured_at' not found in data for station {station_id}.")
        return None
    
    # Ensure temperature is numeric.
    if "t" in df.columns:
        df["t"] = pd.to_numeric(df["t"], errors="coerce")
    else:
        print(f"'t' not found in data for station {station_id}.")
        return None
    
    # Return the DataFrame with the two columns.
    return df[["measured_at", "t"]].copy()

# For the target date, we want March 1, 2025.
target_date = inputs["target_date"]

# Get the list of station dictionaries for Nouvelle-Aquitaine
stations = inputs["stations_for_region"]

# List to collect DataFrames for each station.
dfs = []

for station in stations:
    station_id = station["ID"]
    print(f"Fetching data for station {station['Nom']} (ID: {station_id}) for {target_date}...")
    df_station = fetch_station_measures(station_id, target_date)
    if df_station is not None:
        # Rename the temperature column to include the station name (or ID) for clarity.
        df_station.rename(columns={"t": f"t_{station_id}"}, inplace=True)
        dfs.append(df_station)

if dfs:
    # Merge DataFrames from all stations on the "measured_at" timestamp.
    # We assume the timestamps are nearly aligned. We'll do an outer join and then align.
    df_merged = dfs[0]
    for df in dfs[1:]:
        df_merged = pd.merge_asof(df_merged.sort_values("measured_at"),
                                  df.sort_values("measured_at"),
                                  on="measured_at",
                                  tolerance=pd.Timedelta("15min"),
                                  direction="nearest")
    
    # Filter the merged DataFrame to only include rows that are common (or fill missing values as needed)
    # For simplicity, we drop rows with any missing temperature values:
    df_merged.dropna(inplace=True)
    
    # Compute the regional average temperature (row-wise average across the temperature columns).
    temp_columns = [col for col in df_merged.columns if col.startswith("t_")]
    df_merged["t_avg"] = df_merged[temp_columns].mean(axis=1)
    
    # Convert from Kelvin to Celsius if necessary:
    df_merged["t_avg"] = df_merged["t_avg"] - 273.15
        
else:
    print("No weather data retrieved for the region.")


Fetching data for station CLERMONT-FD (ID: 07460) for 2025-03-26...
Fetching data for station LE PUY-LOUDES (ID: 07471) for 2025-03-26...
Fetching data for station LYON-ST EXUPERY (ID: 07481) for 2025-03-26...
Fetching data for station MONTELIMAR (ID: 07577) for 2025-03-26...


In [8]:
df_merged.describe

<bound method NDFrame.describe of             measured_at  t_07460  t_07471  t_07481  t_07577   t_avg
0   2025-03-26 00:00:00   282.85   279.85   283.95   285.55   9.900
1   2025-03-26 03:00:00   281.85   279.35   282.35   284.75   8.925
2   2025-03-26 06:00:00   281.35   279.15   280.95   284.35   8.300
3   2025-03-26 09:00:00   282.65   280.35   283.05   285.95   9.850
4   2025-03-26 12:00:00   284.45   281.95   286.35   287.75  11.975
..                  ...      ...      ...      ...      ...     ...
96  2025-04-07 00:00:00   283.85   276.25   281.15   286.75   8.850
97  2025-04-07 03:00:00   275.65   274.05   278.45   284.55   5.025
98  2025-04-07 06:00:00   276.65   273.75   277.25   282.35   4.350
99  2025-04-07 09:00:00   285.25   283.55   282.65   286.85  11.425
100 2025-04-07 12:00:00   290.35   288.95   287.75   291.35  16.450

[92 rows x 6 columns]>

In [20]:
df_merged.to_csv("temp_{}_{}_hist".format(region_lwrc, date_str), index=False)

### Via CSV read

In [21]:
df_merged = pd.read_csv("temp_{}_{}_hist".format(region_lwrc, date_str), parse_dates=["measured_at"])

### Creating df_t_hist

#### NEW

In [22]:
temp_hist_dates = [((inputs["chosen_day"] - timedelta(days=i)).month, 
                     (inputs["chosen_day"] - timedelta(days=i)).day)
                   for i in range(5, 0, -1)]

df_temp_hist = df_merged[
    (df_merged['measured_at'].dt.year == inputs["chosen_day"].year) &
    (df_merged['measured_at'].dt.month.isin([m for m,d in temp_hist_dates])) &
    (df_merged['measured_at'].dt.day.isin([d for m,d in temp_hist_dates]))
].copy()

# Ensure the measured_at column is a measured_at type and set as index
df_temp_hist["measured_at"] = pd.to_datetime(df_temp_hist["measured_at"])
df_temp_hist.set_index("measured_at", inplace=True)

# Resample the data to 15-minute intervals and interpolate linearly
df_temp_hist = df_temp_hist.resample("15min").interpolate(method="linear")

# Reset the index so that 'measured_at' is again a column
df_temp_hist.reset_index(inplace=True)

# For the temp_hist data DataFrame (df_temp_hist):
df_temp_hist.rename(columns={"measured_at": "Datetime"}, inplace=True)

# Generate tuples for the 4 days prior to the chosen day, excluding the day immediately before
# For example, for chosen_day = 2025-02-25, we want data for Feb 20, 21, 22, and 23.
temp_hist_dates_subset = [((inputs["chosen_day"] - timedelta(days=i)).month,
                             (inputs["chosen_day"] - timedelta(days=i)).day)
                            for i in range(5, 0, -1)]

# This will produce: [(2, 20), (2, 21), (2, 22), (2, 23)]

# Now filter df_temp_hist using these tuples:
df_t_hist = df_temp_hist[
    (df_temp_hist['Datetime'].dt.month.isin([m for m, d in temp_hist_dates_subset])) &
    (df_temp_hist['Datetime'].dt.day.isin([d for m, d in temp_hist_dates_subset]))
].copy()

drop_columns = [f"t_{station['ID']}" for station in inputs["stations_for_region"]]

df_t_hist.drop(columns=drop_columns, inplace=True)
df_t_hist.rename(columns={"t_avg": "t"}, inplace=True)

In [23]:
df_t_hist.describe

<bound method NDFrame.describe of                Datetime      t
0   2025-02-28 00:00:00  4.200
1   2025-02-28 00:15:00  3.975
2   2025-02-28 00:30:00  3.750
3   2025-02-28 00:45:00  3.525
4   2025-02-28 01:00:00  3.300
..                  ...    ...
464 2025-03-04 20:00:00  6.700
465 2025-03-04 20:15:00  6.225
466 2025-03-04 20:30:00  5.750
467 2025-03-04 20:45:00  5.275
468 2025-03-04 21:00:00  4.800

[469 rows x 2 columns]>

### Via Historical Data

In [8]:
df_t_hist = pd.read_csv(r"C:\Users\Henri\Documents\GitHub\Predi_Conso_Elec_Region\Predi_Conso_Elec_Region\data\reg_2025_temperature.csv", parse_dates=["Datetime"])

In [9]:
df_t_hist.head()

,Datetime,Région,t
0,2025-01-01 00:00:00,Auvergne-Rhône-Alpes,-3.566667
1,2025-01-01 00:15:00,Auvergne-Rhône-Alpes,-3.591667
2,2025-01-01 00:30:00,Auvergne-Rhône-Alpes,-3.616667
3,2025-01-01 00:45:00,Auvergne-Rhône-Alpes,-3.641667
4,2025-01-01 01:00:00,Auvergne-Rhône-Alpes,-3.666667


### df_consumption Feature Engineering

In [11]:
# FEATURE ENGINEERING WITH TIME MARKERS
df_consumption['DayOfWeek'] = df_consumption['Datetime'].dt.weekday
df_consumption['IsWeekend'] = df_consumption['DayOfWeek'].isin([5, 6])  # Saturday and Sunday are weekend
df_consumption['HourOfDay'] = df_consumption['Datetime'].dt.hour

df_consumption['IsMorning'] = df_consumption['HourOfDay'].between(6, 11)
df_consumption['IsAfternoon'] = df_consumption['HourOfDay'].between(12, 17)
df_consumption['IsEvening'] = df_consumption['HourOfDay'].between(18, 22)
df_consumption['IsNight'] = (df_consumption['HourOfDay'] >= 23) | (df_consumption['HourOfDay'] <= 5)

df_consumption["Month"] = df_consumption["Datetime"].dt.month
df_consumption["week_of_year"] = df_consumption["Datetime"].dt.isocalendar().week.astype(float)

# Annual Seasonality
df_consumption["day_of_year"] = df_consumption["Datetime"].dt.dayofyear
df_consumption["sin_annual"] = np.sin(2 * np.pi * df_consumption["day_of_year"] / 365.25)
df_consumption["cos_annual"] = np.cos(2 * np.pi * df_consumption["day_of_year"] / 365.25)

# Weekly Seasonality
df_consumption["sin_weekly"] = np.sin(2 * np.pi * df_consumption["week_of_year"] / 52)
df_consumption["cos_weekly"] = np.cos(2 * np.pi * df_consumption["week_of_year"] / 52)

In [12]:
from vacances_scolaires_france import SchoolHolidayDates

def add_holiday_column(df_consumption):
    # 1. Map regions to zones
    df_consumption = df_consumption.copy()  # Work on a copy to avoid modifying the original
    df_consumption["Zone"] = df_consumption["Région"].map(holiday_zones)
    
    # 2. Handle missing zones
    df_consumption["Zone"] = df_consumption["Zone"].fillna("Unknown")
    
    # 3. Precompute holidays for unique date-zone pairs
    date_zones = df_consumption[["Datetime", "Zone"]].drop_duplicates()
    holiday_checker = SchoolHolidayDates()
    
    date_zones["Holiday"] = date_zones.apply(
        lambda x: holiday_checker.is_holiday_for_zone(
            x["Datetime"].date(),
            x["Zone"]
        ) if x["Zone"] != "Unknown" else False,
        axis=1
    )
    
    # 4. Merge back into main DataFrame
    return df_consumption.merge(date_zones, on=["Datetime", "Zone"])

# Add the Holiday column
df_consumption = add_holiday_column(df_consumption)

# Convert to binary (1/0)
df_consumption["Holiday"] = df_consumption["Holiday"].astype(int)

# Drop the Adjusted column to avoid confusion
df_consumption.drop(columns=['Zone'], inplace=True)
def add_time_features(df):
    return df_consumption.assign(
        Month=df_consumption.Datetime.dt.month,
        WeekOfYear=df_consumption.Datetime.dt.isocalendar().week,
    )

# Apply to the ENTIRE dataset first
df_consumption = add_time_features(df_consumption)

In [13]:
df_consumption.head()

,Datetime,Consommation (MW),Région,DayOfWeek,IsWeekend,HourOfDay,IsMorning,IsAfternoon,IsEvening,IsNight,Month,week_of_year,day_of_year,sin_annual,cos_annual,sin_weekly,cos_weekly,Holiday,WeekOfYear
0,2025-01-01,9557.0,Île-de-France,2,False,0,False,False,False,True,1,1.0,1,0.017202,0.999852,0.120537,0.992709,1,1
1,2025-01-01,2793.0,Bretagne,2,False,0,False,False,False,True,1,1.0,1,0.017202,0.999852,0.120537,0.992709,1,1
2,2025-01-01,6478.0,Nouvelle-Aquitaine,2,False,0,False,False,False,True,1,1.0,1,0.017202,0.999852,0.120537,0.992709,1,1
3,2025-01-01,3441.0,Normandie,2,False,0,False,False,False,True,1,1.0,1,0.017202,0.999852,0.120537,0.992709,1,1
4,2025-01-01,2619.0,Centre-Val de Loire,2,False,0,False,False,False,True,1,1.0,1,0.017202,0.999852,0.120537,0.992709,1,1


In [14]:
df_consumption.describe

<bound method NDFrame.describe of                   Datetime  Consommation (MW)                   Région  \
0      2025-01-01 00:00:00             9557.0            Île-de-France   
1      2025-01-01 00:00:00             2793.0                 Bretagne   
2      2025-01-01 00:00:00             6478.0       Nouvelle-Aquitaine   
3      2025-01-01 00:00:00             3441.0                Normandie   
4      2025-01-01 00:00:00             2619.0      Centre-Val de Loire   
...                    ...                ...                      ...   
104827 2025-04-01 23:45:00                NaN     Auvergne-Rhône-Alpes   
104828 2025-04-01 23:45:00                NaN                Normandie   
104829 2025-04-01 23:45:00                NaN  Bourgogne-Franche-Comté   
104830 2025-04-01 23:45:00                NaN      Centre-Val de Loire   
104831 2025-04-01 23:45:00                NaN            Île-de-France   

        DayOfWeek  IsWeekend  HourOfDay  IsMorning  IsAfternoon  IsEvening  \

#### Creating df_elec

In [15]:
df_elec = df_consumption.copy()


In [ ]:
df_elec["Datetime"] = df_elec["Datetime"].dt.tz_localize(None)

In [16]:
df_elec.columns

Index(['Datetime', 'Consommation (MW)', 'Région', 'DayOfWeek', 'IsWeekend',
       'HourOfDay', 'IsMorning', 'IsAfternoon', 'IsEvening', 'IsNight',
       'Month', 'week_of_year', 'day_of_year', 'sin_annual', 'cos_annual',
       'sin_weekly', 'cos_weekly', 'Holiday', 'WeekOfYear'],
      dtype='object')

In [17]:
df_t_hist.columns

Index(['Datetime', 'Région', 't'], dtype='object')

In [18]:
df_t_hist.head()

,Datetime,Région,t
0,2025-01-01 00:00:00,Auvergne-Rhône-Alpes,-3.566667
1,2025-01-01 00:15:00,Auvergne-Rhône-Alpes,-3.591667
2,2025-01-01 00:30:00,Auvergne-Rhône-Alpes,-3.616667
3,2025-01-01 00:45:00,Auvergne-Rhône-Alpes,-3.641667
4,2025-01-01 01:00:00,Auvergne-Rhône-Alpes,-3.666667


In [19]:
len(df_consumption)

104832

In [27]:
print(df_consumption["Région"].value_counts())

Région
Île-de-France                 8736
Bretagne                      8736
Nouvelle-Aquitaine            8736
Normandie                     8736
Centre-Val de Loire           8736
Bourgogne-Franche-Comté       8736
Auvergne-Rhône-Alpes          8736
Provence-Alpes-Côte d'Azur    8736
Occitanie                     8736
Pays de la Loire              8736
Hauts-de-France               8736
Grand Est                     8736
Name: count, dtype: int64


In [28]:
print(df_consumption["Région"].nunique())

12


In [37]:
print(set(df_consumption["Région"].unique()) - set(df_t_hist["Région"].unique()))

{"Provence-Alpes-Côte d'Azur"}


In [20]:
len(df_t_hist)

120806

In [21]:
print(df_t_hist["Région"].value_counts())

Région
Auvergne-Rhône-Alpes          8629
Bourgogne-Franche-Comté       8629
Bretagne                      8629
Centre-Val de Loire           8629
Corse                         8629
Grand Est                     8629
Hauts-de-France               8629
Normandie                     8629
Nouvelle-Aquitaine            8629
Occitanie                     8629
Pays de la Loire              8629
Provence-Alpes-Côte d’Azur    8629
Unkown                        8629
Île-de-France                 8629
Name: count, dtype: int64


In [33]:
df_t_hist = df_t_hist[~df_t_hist["Région"].isin(["Corse", "Unkown"])]

In [34]:
print(df_t_hist["Région"].value_counts())

Région
Auvergne-Rhône-Alpes          8629
Bourgogne-Franche-Comté       8629
Bretagne                      8629
Centre-Val de Loire           8629
Grand Est                     8629
Hauts-de-France               8629
Normandie                     8629
Nouvelle-Aquitaine            8629
Occitanie                     8629
Pays de la Loire              8629
Provence-Alpes-Côte d’Azur    8629
Île-de-France                 8629
Name: count, dtype: int64


In [35]:
print(df_t_hist["Région"].nunique())

12


In [36]:
print(set(df_t_hist["Région"].unique()) - set(df_consumption["Région"].unique()))

{'Provence-Alpes-Côte d’Azur'}


In [38]:
import unicodedata

df_t_hist["Région"] = df_t_hist["Région"].apply(
    lambda x: unicodedata.normalize("NFKD", x)
)


In [39]:
import unicodedata

df_consumption["Région"] = df_consumption["Région"].apply(
    lambda x: unicodedata.normalize("NFKD", x)
)


In [40]:
print(set(df_t_hist["Région"].unique()) - set(df_consumption["Région"].unique()))

{'Provence-Alpes-Côte d’Azur'}


In [43]:
print(set(df_consumption["Région"].unique()) - set(df_t_hist["Région"].unique()))

set()


In [42]:
df_t_hist["Région"] = df_t_hist["Région"].str.replace("’", "'", regex=False)


In [44]:
print(df_t_hist["Région"].value_counts())

Région
Auvergne-Rhône-Alpes          8629
Bourgogne-Franche-Comté       8629
Bretagne                       8629
Centre-Val de Loire            8629
Grand Est                      8629
Hauts-de-France                8629
Normandie                      8629
Nouvelle-Aquitaine             8629
Occitanie                      8629
Pays de la Loire               8629
Provence-Alpes-Côte d'Azur    8629
Île-de-France                 8629
Name: count, dtype: int64


In [45]:
df_elec = df_consumption.copy()


In [46]:
df_elec["Datetime"] = df_elec["Datetime"].dt.tz_localize(None)

### Creating cons_temp_df

In [47]:
# Merge using merge_asof to match nearest timestamps within a tolerance.
cons_temp_df = pd.merge(df_elec, df_t_hist, on=["Datetime", "Région"], how="inner")

In [48]:
cons_temp_df.describe

<bound method NDFrame.describe of                   Datetime  Consommation (MW)                 Région  \
0      2025-01-01 00:00:00             9557.0         Île-de-France   
1      2025-01-01 00:00:00             2793.0               Bretagne   
2      2025-01-01 00:00:00             6478.0     Nouvelle-Aquitaine   
3      2025-01-01 00:00:00             3441.0              Normandie   
4      2025-01-01 00:00:00             2619.0    Centre-Val de Loire   
...                    ...                ...                    ...   
103543 2025-03-31 21:00:00             4251.0              Occitanie   
103544 2025-03-31 21:00:00             8344.0         Île-de-France   
103545 2025-03-31 21:00:00             1939.0    Centre-Val de Loire   
103546 2025-03-31 21:00:00             6817.0  Auvergne-Rhône-Alpes   
103547 2025-03-31 21:00:00             2912.0              Normandie   

        DayOfWeek  IsWeekend  HourOfDay  IsMorning  IsAfternoon  IsEvening  \
0               2      

### Save Dataset

In [49]:
# Save new merged dataset
import os

# Save the dataset as a CSV file
cons_temp_df.to_csv("cons_temp_2025.csv", index=False)

# Get the full directory of the saved file
file_path = os.path.abspath("cons_temp_2025.csv")
print(f"File saved at: {file_path}")

File saved at: c:\Users\Henri\Documents\GitHub\Predi_Conso_Elec_Region\Predi_Conso_Elec_Region\notebooks\cons_temp_2025.csv


## Defining Actual Cons

#### Via API Call

In [30]:
# Define the endpoint URL for the eco2mix-regional-tr dataset
url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr/records"

# Build the query parameters using a 'where' filter.
params = {
    "limit": 100,
    "where": inputs["consumption_where_2"],
    "select": "date,heure,date_heure,libelle_region,consommation",
    "timezone": "UTC",
    "include_links": "false",
    "include_app_metas": "false"
}

# Set the request headers
headers = {
    "accept": "application/json; charset=utf-8"
}

# Make the GET request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    consumption_json = response.json()  # Convert response to JSON dictionary
    print(json.dumps(consumption_json, indent=2, ensure_ascii=False))
else:
    print(f"Error: HTTP {response.status_code}")
    print(response.text)

{
  "total_count": 288,
  "results": [
    {
      "date": "2025-03-04",
      "heure": "02:15",
      "date_heure": "2025-03-04T01:15:00+00:00",
      "libelle_region": "Bourgogne-Franche-Comté",
      "consommation": 2472
    },
    {
      "date": "2025-03-04",
      "heure": "04:45",
      "date_heure": "2025-03-04T03:45:00+00:00",
      "libelle_region": "Bourgogne-Franche-Comté",
      "consommation": 2366
    },
    {
      "date": "2025-03-04",
      "heure": "05:00",
      "date_heure": "2025-03-04T04:00:00+00:00",
      "libelle_region": "Bourgogne-Franche-Comté",
      "consommation": 2415
    },
    {
      "date": "2025-03-04",
      "heure": "06:00",
      "date_heure": "2025-03-04T05:00:00+00:00",
      "libelle_region": "Bourgogne-Franche-Comté",
      "consommation": 2593
    },
    {
      "date": "2025-03-04",
      "heure": "06:45",
      "date_heure": "2025-03-04T05:45:00+00:00",
      "libelle_region": "Bourgogne-Franche-Comté",
      "consommation": 2857
    },
 

In [31]:
# Suppose you have loaded your JSON into a variable called consumption_json
# For example:
# consumption_json = json.loads(your_json_string)

# Create a DataFrame from the "results" list
df_consumption_actual = pd.DataFrame(consumption_json["results"])

# Use the "date_heure" field as the timestamp. Convert it to datetime.
df_consumption_actual["datetime"] = pd.to_datetime(df_consumption_actual["date_heure"])

# Select only the needed columns (e.g., datetime, consommation, and optionally region)
df_consumption_actual = df_consumption_actual[["datetime", "consommation", "libelle_region"]].copy()

# For the consumption data DataFrame (df_consumption_actual):
df_consumption_actual.rename(columns={"datetime": "Datetime", "consommation": "Consommation (MW)", "libelle_region" : "Région"}, inplace=True)

# Sort by datetime
df_consumption_actual.sort_values("Datetime", inplace=True)

# Then remove the timezone information (e.g., "+00:00")
df_consumption_actual["Datetime"] = df_consumption_actual["Datetime"].dt.tz_convert("Europe/Paris")

df_consumption_actual.set_index("Datetime", inplace=True)

# Resample the data to 15-minute intervals and interpolate linearly
df_consumption_actual = df_consumption_actual.resample("15min").interpolate(method="linear")

# Reset the index so that 'Datetime' is again a column
df_consumption_actual.reset_index(inplace=True)

# Fill NaN values in "Région" column using forward fill
df_consumption_actual["Région"] = df_consumption_actual["Région"].ffill()

C:\Users\Henri\AppData\Local\Temp\ipykernel_8332\2132051619.py:26: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_consumption_actual = df_consumption_actual.resample("15min").interpolate(method="linear")


In [32]:
df_consumption_actual.describe

<bound method NDFrame.describe of                      Datetime  Consommation (MW)                   Région
0   2025-03-04 00:30:00+01:00            2421.00  Bourgogne-Franche-Comté
1   2025-03-04 00:45:00+01:00            2398.75  Bourgogne-Franche-Comté
2   2025-03-04 01:00:00+01:00            2376.50  Bourgogne-Franche-Comté
3   2025-03-04 01:15:00+01:00            2354.25  Bourgogne-Franche-Comté
4   2025-03-04 01:30:00+01:00            2332.00  Bourgogne-Franche-Comté
..                        ...                ...                      ...
278 2025-03-06 22:00:00+01:00            2306.20  Bourgogne-Franche-Comté
279 2025-03-06 22:15:00+01:00            2300.40  Bourgogne-Franche-Comté
280 2025-03-06 22:30:00+01:00            2294.60  Bourgogne-Franche-Comté
281 2025-03-06 22:45:00+01:00            2288.80  Bourgogne-Franche-Comté
282 2025-03-06 23:00:00+01:00            2283.00  Bourgogne-Franche-Comté

[283 rows x 3 columns]>

In [33]:
df_consumption_actual["Datetime"] = df_consumption_actual["Datetime"].dt.tz_localize(None)

#### Via saved JSON Read

In [34]:
with open ("actual_cons_{}_{}.json".format(region_lwrc, date_str), "r", encoding="utf-8") as f: df_consumption_actual = json.load(f)

In [35]:
df_consumption_actual = pd.json_normalize(df_consumption_actual, record_path=["results"])

In [36]:

# Use the "date_heure" field as the timestamp. Convert it to datetime.
df_consumption_actual["datetime"] = pd.to_datetime(df_consumption_actual["date_heure"])

# Select only the needed columns (e.g., datetime, consommation, and optionally region)
df_consumption_actual = df_consumption_actual[["datetime", "consommation", "libelle_region"]].copy()

# For the consumption data DataFrame (df_consumption_actual):
df_consumption_actual.rename(columns={"datetime": "Datetime", "consommation": "Consommation (MW)", "libelle_region" : "Région"}, inplace=True)

# Sort by datetime
df_consumption_actual.sort_values("Datetime", inplace=True)

# Then remove the timezone information (e.g., "+00:00")
df_consumption_actual["Datetime"] = df_consumption_actual["Datetime"].dt.tz_convert("Europe/Paris")

df_consumption_actual.set_index("Datetime", inplace=True)

# Resample the data to 15-minute intervals and interpolate linearly
df_consumption_actual = df_consumption_actual.resample("15min").interpolate(method="linear")

# Reset the index so that 'Datetime' is again a column
df_consumption_actual.reset_index(inplace=True)

# Fill NaN values in "Région" column using forward fill
df_consumption_actual["Région"] = df_consumption_actual["Région"].ffill()

C:\Users\Henri\AppData\Local\Temp\ipykernel_8332\2156597977.py:19: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_consumption_actual = df_consumption_actual.resample("15min").interpolate(method="linear")


In [37]:
df_consumption_actual["Datetime"] = df_consumption_actual["Datetime"].dt.tz_localize(None)

### Defining df_elec_actual

In [38]:
df_elec_actual = df_consumption_actual[df_consumption_actual['Datetime'].dt.day == inputs["chosen_day"].day].copy()

In [39]:
df_elec_actual.tail()

,Datetime,Consommation (MW),Région
185,2025-03-05 22:45:00,2498.0,Bourgogne-Franche-Comté
186,2025-03-05 23:00:00,2468.4,Bourgogne-Franche-Comté
187,2025-03-05 23:15:00,2438.8,Bourgogne-Franche-Comté
188,2025-03-05 23:30:00,2409.2,Bourgogne-Franche-Comté
189,2025-03-05 23:45:00,2379.6,Bourgogne-Franche-Comté


### Save File

In [40]:
df_elec_actual.to_csv("actual_cons_{}_{}.csv".format(region_lwrc, date_str), index=False)

## Other Commands

In [ ]:
dt_start = inputs["first_row"] - inputs["deltatime"]

In [ ]:
dt_end = inputs["last_row"] - inputs["deltatime"]

In [ ]:
df_filtered = cons_temp_df[(cons_temp_df['Datetime'] >= dt_start) & (cons_temp_df['Datetime'] <= dt_end)]

In [ ]:
print(inputs["last_row"])

2025-03-05 23:45:00


In [ ]:
print(dt_end)

2025-03-04 11:45:00


In [ ]:
# Create a mask to identify rows on the last day (D-1)
mask_last_day = (df_temp_hist['Datetime'].dt.month == last_month) & (df_temp_hist['Datetime'].dt.day == last_day)

# Among those, define a cutoff: we want to drop rows with hour >=6 or with hour == 5 and minute > 45.
mask_cutoff = mask_last_day & (
    ((df_temp_hist['Datetime'].dt.hour == 5) & (df_temp_hist['Datetime'].dt.minute > 45)) |
    (df_temp_hist['Datetime'].dt.hour >= 6)
)

# Drop those rows
df_temp_hist = df_temp_hist[~mask_cutoff].copy()


In [ ]:
# 3. For the last day (D-1), keep only data before (or at) 05:45.
# Get the tuple for the last day:
last_month, last_day = cons_hist_dates[-1]


C:\Users\Henri\AppData\Local\Temp\ipykernel_444\2684201633.py:8: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_cons_hist = df_cons_hist.resample("15min").interpolate(method="linear")
